In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
print("prueba Sergio")

prueba Sergio


In [ ]:
#víctor

In [3]:
archivo="players_22.csv"
df=pd.read_csv(archivo)

NameError: name 'pd' is not defined